## Prerequisites

Unfortunatelly Google Cloud and AWS are not available in my country, so I will use Yandex.Cloud. I will use ClickHouse instead of BigQuery.

### Load data to S3 Bucket

Create deployments:
```bash
prefect deployment build flows/etl_web_to_gcs.py:etl_web_to_gcs_multiple \
    -n etl_web_to_gcs_multiple_fhv_2019 \
    --params '{"months": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], "year": 2019, "color": "fhv"}' \
    --output 'etl_web_to_gcs_multiple_fhv_2019-deployment.yaml'

prefect deployment apply etl_web_to_gcs_multiple_fhv_2019-deployment.yaml
```

Then run them in Prefect UI: http://localhost:4200/deployments

In [1]:
import os

from dotenv import load_dotenv

load_dotenv()  # load credentials

True

### Check connection with ClickHouse

In [2]:
from clickhouse_driver import Client

client = Client(host=os.environ['CH_HOST'],
                user=os.environ['CH_USER'],
                password=os.environ['CH_PASS'],
                database='default',
                secure=True)

client.execute('SELECT version()')

[('22.8.13.20',)]

In [3]:
!wget https://storage.yandexcloud.net/cloud-certs/CA.pem -O YandexCA.crt
!chmod 655 YandexCA.crt

--2023-02-12 19:57:49--  https://storage.yandexcloud.net/cloud-certs/CA.pem
Распознаётся storage.yandexcloud.net (storage.yandexcloud.net)… 213.180.193.243
Подключение к storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 3579 (3,5K) [application/x-x509-ca-cert]
Сохранение в: «YandexCA.crt»

YandexCA.crt        100%[===================>]   3,50K  --.-KB/s    за 0s      

2023-02-12 19:57:49 (341 MB/s) - «YandexCA.crt» сохранён [3579/3579]



In [4]:
import pandas as pd
import sqlalchemy

engine = sqlalchemy.create_engine(
    f'clickhouse+http://{os.environ["CH_USER"]}:{os.environ["CH_PASS"]}@{os.environ["CH_HOST"]}:8443/default'
    '?protocol=https&verify=YandexCA.crt'
)

pd.read_sql('SELECT version()', con=engine)

,version()
0,22.8.13.20


In [5]:
def drop_table(table):
    return f'DROP TABLE IF EXISTS {table}'
    

client.execute(drop_table('fhv_tripdata_csv'))
client.execute(drop_table('fhv_tripdata_table'))
client.execute(drop_table('fhv_tripdata_table_partitioned'))

[]

### Sanity Check Table

In [6]:
df_check = pd.read_csv(
    'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-02.csv.gz',
    parse_dates=[
        'pickup_datetime',
        'dropOff_datetime',
    ],
)

df_check = df_check[df_check['dispatching_base_num'] == 'B03022']
df_check.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
13167,B03022,2019-02-01 07:24:53,2019-02-01 07:42:53,264.0,265.0,NaN,B03022
18768,B03022,2019-02-01 08:01:31,2019-02-01 08:32:10,264.0,265.0,NaN,B03022
18769,B03022,2019-02-01 08:32:09,2019-02-01 09:35:09,264.0,265.0,1.0,B03022
18770,B03022,2019-02-01 08:57:13,2019-02-01 09:14:36,264.0,265.0,2.0,B03022
23864,B03022,2019-02-01 09:17:34,2019-02-01 09:23:38,264.0,265.0,1.0,B03022


### Create External Table

https://clickhouse.com/docs/ru/engines/table-engines/integrations/s3/

In [7]:
sql_query = f'''
CREATE TABLE IF NOT EXISTS default.fhv_tripdata_csv
(
    dispatching_base_num              String,
    pickup_datetime                   DateTime,
    dropOff_datetime                  DateTime,
    PULocationID                      Nullable(Int32),
    DOLocationID                      Nullable(Int32),
    SR_Flag                           Nullable(Int8),
    Affiliated_base_number            String
)
ENGINE = S3(
    'https://storage.yandexcloud.net/de-bucket-dev/data/fhv/fhv_tripdata_2019-*.cvs.gz',
    {os.environ["YA_S3_KEY"]},
    {os.environ["YA_S3_TOKEN"]},
    'CSV',
    'gzip'
)
SETTINGS input_format_csv_skip_first_lines = 1;
'''

client.execute(sql_query)

[]

In [8]:
pd.read_sql('''
select * from default.fhv_tripdata_csv
where true
    and toDate(pickup_datetime) = toDate('2019-02-01')
    and dispatching_base_num = 'B03022'
limit 5
''', con=engine)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PULocationID,DOLocationID,SR_Flag,Affiliated_base_number
0,B03022,2019-02-01 07:24:53,2019-02-01 07:42:53,264,265,NaN,B03022
1,B03022,2019-02-01 08:01:31,2019-02-01 08:32:10,264,265,NaN,B03022
2,B03022,2019-02-01 08:32:09,2019-02-01 09:35:09,264,265,1.0,B03022
3,B03022,2019-02-01 08:57:13,2019-02-01 09:14:36,264,265,2.0,B03022
4,B03022,2019-02-01 09:17:34,2019-02-01 09:23:38,264,265,1.0,B03022


In [9]:
# client.execute(drop_table('fhv_tripdata_csv'))

### Create ClickHouse Table

- https://clickhouse.com/docs/ru/engines/table-engines/mergetree-family/mergetree
- https://clickhouse.com/docs/ru/engines/table-engines/special/memory

In [10]:
sql_query = '''
CREATE TABLE IF NOT EXISTS default.fhv_tripdata_table
ENGINE = MergeTree()
ORDER BY pickup_datetime -- you can't create table without 'order by' statement
AS ( SELECT * FROM default.fhv_tripdata_csv )
'''

client.execute(sql_query)

[]

In [11]:
pd.read_sql('''
select * from default.fhv_tripdata_table
where true
    and toDate(pickup_datetime) = toDate('2019-02-01')
    and dispatching_base_num = 'B03022'
limit 5
''', con=engine)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PULocationID,DOLocationID,SR_Flag,Affiliated_base_number
0,B03022,2019-02-01 07:24:53,2019-02-01 07:42:53,264,265,NaN,B03022
1,B03022,2019-02-01 08:01:31,2019-02-01 08:32:10,264,265,NaN,B03022
2,B03022,2019-02-01 08:32:09,2019-02-01 09:35:09,264,265,1.0,B03022
3,B03022,2019-02-01 08:57:13,2019-02-01 09:14:36,264,265,2.0,B03022
4,B03022,2019-02-01 09:00:12,2019-02-01 09:29:43,264,265,1.0,B03022


In [12]:
# client.execute(drop_table('fhv_tripdata_table'))

### Create ClickHouse Table (Partitioned and Clustered)

Possibly we will have to use more memory to create table.

In [13]:
sql_query = '''
CREATE TABLE IF NOT EXISTS default.fhv_tripdata_table_partitioned
ENGINE = MergeTree()
ORDER BY Affiliated_base_number
PARTITION BY toYYYYMMDD(pickup_datetime)
AS ( SELECT * FROM default.fhv_tripdata_table )
'''

client.execute(sql_query)

[]

In [14]:
pd.read_sql('''
select * from default.fhv_tripdata_table_partitioned
where true
    and toDate(pickup_datetime) = toDate('2019-02-01')
    and dispatching_base_num = 'B03022'
limit 5
''', con=engine)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PULocationID,DOLocationID,SR_Flag,Affiliated_base_number
0,B03022,2019-02-01 07:24:53,2019-02-01 07:42:53,264,265,NaN,B03022
1,B03022,2019-02-01 08:01:31,2019-02-01 08:32:10,264,265,NaN,B03022
2,B03022,2019-02-01 08:32:09,2019-02-01 09:35:09,264,265,1.0,B03022
3,B03022,2019-02-01 08:57:13,2019-02-01 09:14:36,264,265,2.0,B03022
4,B03022,2019-02-01 09:00:12,2019-02-01 09:29:43,264,265,1.0,B03022


In [15]:
# client.execute(drop_table('fhv_tripdata_table_partitioned'))

## Tasks

### Question 1

In [16]:
%%time

pd.read_sql('''
select count(*) from default.fhv_tripdata_csv
''', con=engine)

CPU times: user 10.4 ms, sys: 2.96 ms, total: 13.3 ms
Wall time: 19.7 s


,count()
0,43244696


### Question 2

In [17]:
%%time

pd.read_sql('''
select uniq(Affiliated_base_number) from default.fhv_tripdata_csv
''', con=engine)

CPU times: user 11 ms, sys: 3.94 ms, total: 14.9 ms
Wall time: 22 s


,uniq(Affiliated_base_number)
0,3037


In [18]:
%%time

pd.read_sql('''
select count(distinct Affiliated_base_number) from default.fhv_tripdata_csv
''', con=engine)

CPU times: user 13.1 ms, sys: 4.57 ms, total: 17.7 ms
Wall time: 22.8 s


,uniqExact(Affiliated_base_number)
0,3037


In [19]:
%%time

pd.read_sql('''
select uniq(Affiliated_base_number) from default.fhv_tripdata_table
''', con=engine)

CPU times: user 11.9 ms, sys: 3.75 ms, total: 15.6 ms
Wall time: 1.1 s


,uniq(Affiliated_base_number)
0,3037


In [20]:
%%time

pd.read_sql('''
select count(distinct Affiliated_base_number) from default.fhv_tripdata_table
''', con=engine)

CPU times: user 12.1 ms, sys: 4.04 ms, total: 16.1 ms
Wall time: 1.59 s


,uniqExact(Affiliated_base_number)
0,3037


**NOTE:**

It's not possible to estimate memory, that will be used for the query using `fhv_tripdata_csv`, because files are not actually loaded into ClickHouse.

Memory consumption for the query using `fhv_tripdata_table` will be estimated approximately as a size of the table.

### Question 3

In [21]:
%%time

pd.read_sql('''
select count(*)
from default.fhv_tripdata_table
where PULocationID is null and DOLocationID is null
''', con=engine)

CPU times: user 9.63 ms, sys: 3.07 ms, total: 12.7 ms
Wall time: 634 ms


,count()
0,717748


### Question 4

See "Create ClickHouse Table (Partitioned and Clustered)".

### Question 5

In [22]:
%%time

pd.read_sql('''
select count(distinct Affiliated_base_number)
from default.fhv_tripdata_table
where toDate(pickup_datetime) between toDate('2019/03/01') and toDate('2019/03/31')
''', con=engine)

CPU times: user 9.66 ms, sys: 2.7 ms, total: 12.4 ms
Wall time: 101 ms


,uniqExact(Affiliated_base_number)
0,698


In [23]:
%%time

pd.read_sql('''
select count(distinct Affiliated_base_number)
from default.fhv_tripdata_table_partitioned
where toDate(pickup_datetime) between toDate('2019/03/01') and toDate('2019/03/31')
''', con=engine)

CPU times: user 11.2 ms, sys: 3.69 ms, total: 14.9 ms
Wall time: 117 ms


,uniqExact(Affiliated_base_number)
0,698


The second query should be optimized, because the data is already sorted by `Affiliated_base_number`, so this fact can be used for reducing memory consumption in 2nd case.

### Question 6

Data for External Table is stored in Buckets (S3).

### Question 7

It's a good practice always to have **partitioned** tables.